In [3]:
# Libraries you should have installed:"
# twilio
# pandas
# requests
# tqdm

from twilio.rest import Client
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [4]:
# Location for which you want to check the forecast
query = 'London' # !!!

api_key = 'YOUR WEATHERAPI KEY' # !!!

url = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'


API call

In [5]:
response = requests.get(url).json()

In [7]:
response

{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.52,
  'lon': -0.11,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1697306825,
  'localtime': '2023-10-14 19:07'},
 'current': {'last_updated_epoch': 1697306400,
  'last_updated': '2023-10-14 19:00',
  'temp_c': 9.0,
  'temp_f': 48.2,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 8.1,
  'wind_kph': 13.0,
  'wind_degree': 280,
  'wind_dir': 'W',
  'pressure_mb': 1020.0,
  'pressure_in': 30.12,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 81,
  'cloud': 25,
  'feelslike_c': 7.4,
  'feelslike_f': 45.2,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 9.4,
  'gust_kph': 15.2},
 'forecast': {'forecastday': [{'date': '2023-10-14',
    'date_epoch': 1697241600,
    'day': {'maxtemp_c': 13.8,
     'maxtemp_f': 56.8,
     'mintemp_c': 9.1,
     '

Obtaining the keys from the JSON: this is a good practice when working with JSON to make navigation easier

In [6]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

The same inside forecastday and its value 0, which is the hour (after exploring the JSON, we arrived at this deduction)

In [8]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [9]:
response['forecast']['forecastday'][0]['hour']

[{'time_epoch': 1697238000,
  'time': '2023-10-14 00:00',
  'temp_c': 10.5,
  'temp_f': 50.8,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code': 1000},
  'wind_mph': 12.5,
  'wind_kph': 20.2,
  'wind_degree': 294,
  'wind_dir': 'WNW',
  'pressure_mb': 1008.0,
  'pressure_in': 29.78,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 75,
  'cloud': 2,
  'feelslike_c': 8.0,
  'feelslike_f': 46.3,
  'windchill_c': 8.0,
  'windchill_f': 46.3,
  'heatindex_c': 10.5,
  'heatindex_f': 50.8,
  'dewpoint_c': 6.2,
  'dewpoint_f': 43.2,
  'will_it_rain': 0,
  'chance_of_rain': 0,
  'will_it_snow': 0,
  'chance_of_snow': 0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'gust_mph': 17.8,
  'gust_kph': 28.7,
  'uv': 1.0},
 {'time_epoch': 1697241600,
  'time': '2023-10-14 01:00',
  'temp_c': 10.2,
  'temp_f': 50.4,
  'is_day': 0,
  'condition': {'text': 'Clear',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
   'code'

The final 0 is 0:00 AM, as we have observed that it has 24 segments within the hour

In [10]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1697238000,
 'time': '2023-10-14 00:00',
 'temp_c': 10.5,
 'temp_f': 50.8,
 'is_day': 0,
 'condition': {'text': 'Clear',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
  'code': 1000},
 'wind_mph': 12.5,
 'wind_kph': 20.2,
 'wind_degree': 294,
 'wind_dir': 'WNW',
 'pressure_mb': 1008.0,
 'pressure_in': 29.78,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 75,
 'cloud': 2,
 'feelslike_c': 8.0,
 'feelslike_f': 46.3,
 'windchill_c': 8.0,
 'windchill_f': 46.3,
 'heatindex_c': 10.5,
 'heatindex_f': 50.8,
 'dewpoint_c': 6.2,
 'dewpoint_f': 43.2,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 17.8,
 'gust_kph': 28.7,
 'uv': 1.0}

We see in the response that the 'time' field contains the time, to view it:

In [11]:
response['forecast']['forecastday'][0]['hour'][0]['time']

'2023-10-14 00:00'

As we want to split it, we use 'split' since it is a string, the '0' in square brackets at the end is to indicate that we want the first position of the split

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0] # Date (in yyyy - mm - dd format)

'2023-10-14'

To get the hour, we take the second position (1), perform a new split indicating that we want to separate by colons ':', and we keep position 0 (the hour)

In [13]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

We convert it to an integer, so we do the same by adding int

In [14]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]) # Time of day (integer)

1

Now we want to focus on the alerts that it may rain, we explore 'condition' within 'hours

In [15]:
response['forecast']['forecastday'][0]['hour'][0]['condition']

{'text': 'Clear',
 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png',
 'code': 1000}

In [16]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text'] # Weather condition

'Clear'

Now the temperature

In [18]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c'] # Temperature in ºC

10.5

To find out whether it will rain or not, there is the 'will it rain' field

In [19]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain'] # Will it rain? 0 is no, 1 is likely

0

In [20]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain'] # probability of rain

0

# Dataframe creation

In [43]:
def get_forecast(response,i):

    date = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hour = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condition = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    chance_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return date,hour,condition,tempe,rain,chance_rain

In [44]:
data = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    data.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 23984.58it/s]


In [45]:
def create_df(data):

    col = ['Date','Hour','Condition','Temperature','Rain','Chance_of_rain']
    df = pd.DataFrame(data,columns=col)
    df = df.sort_values(by = 'Hour',ascending = True)

    df_rain = df[(df['Rain']==1) & (df['Time']>=6) & (df['Time']<= 22)]
    df_rain = df_rain[['Hour','Condition']]
    df_rain.set_index('Hour', inplace = True)

    return df_rain

In [46]:
col = ['Date','Hour','Condition','Temperature','Rain','Chance_of_rain']
df = pd.DataFrame(data,columns=col)
#df = df.sort_values(by='Hour',ascending = True)
df

,Date,Hour,Condition,Temperature,Rain,Chance_of_rain
0,2023-10-14,0,Clear,10.5,0,0
1,2023-10-14,1,Clear,10.2,0,0
2,2023-10-14,2,Clear,10.0,0,0
3,2023-10-14,3,Clear,9.8,0,0
4,2023-10-14,4,Clear,9.7,0,0
5,2023-10-14,5,Clear,9.4,0,0
6,2023-10-14,6,Clear,9.3,0,0
7,2023-10-14,7,Clear,9.3,0,0
8,2023-10-14,8,Partly cloudy,10.0,0,0
9,2023-10-14,9,Patchy rain possible,11.2,1,72


We keep the hours when it rains between 6 AM and 10 PM and their conditions. We sort by hour

In [51]:
df_rain =  df[(df['Rain']==1) & (df['Hour']>=6) & (df['Hour']<= 22)]
df_rain = df_rain[['Hour','Condition']]
df_rain.set_index('Hour', inplace = True)
df_rain

,Condition
Hour,
9,Patchy rain possible
12,Patchy rain possible
13,Patchy rain possible
14,Patchy rain possible
15,Patchy rain possible


# SMS Message from Twilio

In [64]:
data[0][0]

'2023-10-14'

In [66]:
time.sleep(2)
account_sid = 'YOUR TWILIO SID' # !!!
auth_token = 'YOUR TWILIO TOKEN' # !!!

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                    body='\nHi! \n\n\nToday\'s '+ data[0][0] + ' rain forecast in ' + query +' is:\n\n\n' + str(df_rain),
                    from_='YOUR TWILIO NUMBER', # !!!
                    to='YOUR NUMBER' # !!!
                 )

print('Message Sent ' + message.sid)

Message Sent SMd21303fb605fc6305c66b949b8ab3bda
